In [1]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertForQuestionAnswering
import gc

In [2]:
# tokeniser = BertTokenizer.from_pretrained(model_dir)  # Add specific options if needed
model = BertForQuestionAnswering.from_pretrained('bert-base-chinese')
tokeniser = BertTokenizer.from_pretrained('bert-base-chinese')

model.eval()

if torch.cuda.is_available():

    dirve ='cuda'
else:
    dirve ='cpu'
print('use ',dirve)
model.to(dirve)


use  cuda


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, elementwis

In [13]:
#释放内存
# del model
# gc.collect()

44

In [14]:
model

NameError: name 'model' is not defined

In [3]:
q='狗喜欢吃骨头吗'

a='狗当然喜欢吃骨头了'

len_line_1 = len(tokeniser.tokenize(q))
len_line_2 = len(tokeniser.tokenize(a))

text = q + ' ' + a

#         print('text',text)
tokenized_text = tokeniser.tokenize(text)
#         print('tokenized_text',tokenized_text)

indexed_tokens = tokeniser.convert_tokens_to_ids(tokenized_text)

segments_ids = ([0] * len_line_1) + ([1] * len_line_2)


tokens_tensor = torch.tensor([indexed_tokens]).to(dirve)
segments_tensors = torch.tensor([segments_ids]).to(dirve)
#         print('tokens_tensor',tokens_tensor)
#         print('segments_tensors',segments_tensors)
#这里使用模型进行预测
batch_start_logits, batch_end_logits = model(tokens_tensor, segments_tensors)
print('batch_start_logits',batch_start_logits)
print('batch_end_logits',batch_end_logits)

batch_start_logits tensor([[ 0.2047,  0.1758,  0.0549,  0.2017,  0.2199, -0.2765, -0.0321,  0.0323,
          0.1105, -0.0333, -0.2234, -0.3382,  0.1463,  0.0513, -0.4756, -0.2364]],
       device='cuda:0', grad_fn=<SqueezeBackward1>)
batch_end_logits tensor([[0.6987, 1.1249, 1.0258, 1.1719, 0.8917, 0.8523, 1.1868, 0.7548, 1.3291,
         0.5498, 0.8072, 0.3512, 0.6872, 0.5210, 0.7135, 0.9351]],
       device='cuda:0', grad_fn=<SqueezeBackward1>)


In [4]:
def get_best_indexes(logits, n_best_size):
  """Get the n-best logits from a list."""
  index_and_score = sorted(enumerate(logits), key=lambda x: x[1], reverse=True)

  best_indexes = []
  for i in range(len(index_and_score)):
    if i >= n_best_size:
      break
    best_indexes.append(index_and_score[i][0])
  return best_indexes

In [5]:
n_best_size = 20

In [6]:
start_indexes = get_best_indexes(batch_start_logits, n_best_size)
end_indexes = get_best_indexes(batch_end_logits, n_best_size)
print('start_indexes',start_indexes)
print('end_indexes',end_indexes)

start_indexes [0]
end_indexes [0]


In [7]:
batch_start_logits[0, 0]

tensor(0.2047, device='cuda:0', grad_fn=<SelectBackward>)